In [ ]:
# Gerekli kütüphaneleri yükleme
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener matplotlib openpyxl TvDatafeed

import pandas as pd
import matplotlib.pyplot as plt
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols

# TradingView API bağlantısı
tv = TvDatafeed()

# Tüm BIST hisse senetlerini alma
print("BIST hisse senetleri çekiliyor...")
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Kullanıcıdan zaman dilimini seçme
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("\nLütfen bir zaman dilimi seçin (rakam girilecek):")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choice = input("Zaman dilimini seçin (1-9): ")
    interval_name, interval = intervals.get(choice, ('4 Saat', Interval.in_4_hour))
    return interval_name, interval

# Hacim spike kontrol fonksiyonu (Normalize veriler için)
def volume_spike_check(normalized_data, spike_threshold=2.5):
    # 20 periyotluk hacim ortalaması ekleme
    normalized_data['Volume_Avg'] = normalized_data['volume'].rolling(window=20).mean()

    # Son bar
    last_bar = normalized_data.iloc[-1]
    second_last_bar = normalized_data.iloc[-2]

    # Hacim spike kontrolü
    last_bar_spike = last_bar['volume'] > (last_bar['Volume_Avg'] * spike_threshold)

    # Fiyat hareketi analizi
    if last_bar_spike:
        if last_bar['close'] > second_last_bar['close']:
            return 'Pozitif Spike'
        elif last_bar['close'] < second_last_bar['close']:
            return 'Negatif Spike'
        else:
            return 'Yatay'
    return 'No Spike'

# Normalize edilmiş verilerle hacim spike taraması
def normalized_volume_spike_scan(Hisseler, interval):
    print("XU100 verileri çekiliyor...")
    # XU100 verilerini alma
    xu100_data = tv.get_hist(symbol='XU100', exchange='BIST', interval=interval, n_bars=500)
    xu100_data = xu100_data.reset_index()

    print("Hisse taraması başlatıldı...\n")
    positive_spike_list = []
    negative_spike_list = []

    for hisse in Hisseler:
        try:
            # Hisse verisini alma
            data = tv.get_hist(symbol=hisse, exchange='BIST', interval=interval, n_bars=500)
            data = data.reset_index()

            # Normalize edilmiş veriler oluşturma
            normalized_data = data.copy()
            for column in ['open', 'high', 'low', 'close', 'volume']:
                normalized_data[column] = 100 * (data[column] / xu100_data[column])

            # Hacim spike kontrolü
            spike_status = volume_spike_check(normalized_data)

            # Sonuçları listeye ekleme
            if spike_status == 'Pozitif Spike':
                print(f"{hisse} - Durum: Pozitif Spike")
                positive_spike_list.append(f"{hisse} - Pozitif Spike")
            elif spike_status == 'Negatif Spike':
                print(f"{hisse} - Durum: Negatif Spike")
                negative_spike_list.append(f"{hisse} - Negatif Spike")

        except Exception as e:
            print(f"{hisse} için veri alınırken hata oluştu: {e}")

    return positive_spike_list, negative_spike_list

# Kullanıcıdan zaman dilimi seçimi al
interval_name, interval = get_interval_choice()

# Taramayı başlat
positive_spike_results, negative_spike_results = normalized_volume_spike_scan(Hisseler, interval)

# Tarama sonuçlarını göster
if positive_spike_results or negative_spike_results:
    print("\nPozitif Volume Spike Tespit Edilen Hisseler:")
    for result in positive_spike_results:
        print(result)

    print("\nNegatif Volume Spike Tespit Edilen Hisseler:")
    for result in negative_spike_results:
        print(result)
else:
    print("Volume spike tespit edilen hisse bulunamadı.")


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-1tufopbs
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-1tufopbs
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


ImportError: cannot import name 'get_all_symbols' from 'tradingview_screener' (/usr/local/lib/python3.10/dist-packages/tradingview_screener/__init__.py)